![ADSA Logo](http://i.imgur.com/BV0CdHZ.png?2 \\\"ADSA Logo\\\")

# ADSA Workshop 4 - Introduction to NoSQL and MongoDB

In this code workshop, we will cover the basics of MongoDB. We will first store a subset of Yelp's Academic Dataset (from 1.32 GB to approx. 11 MB). This subset contains all reviews that have the word `mango` in them, and all the businesses these reviews are of.

First we will import pymongo and set up a Mongo Client that will create a connection to interface with the databases we have.

In [1]:
import pymongo

# Connection to Mongo DB
try:
    conn = pymongo.MongoClient()
    print 'Connected successfully!'
except pymongo.errors.ConnectionFailure, e:
    print 'Could not connect to MongoDB: {0}'.format(e)
    
conn

Connected successfully!


MongoClient('localhost', 27017)

## Inserting Data into a MongoDB Database

Let's create a database called `yelp_db` through our connection.

In [2]:
#conn.drop_database('yelp_db')

yelp_db = conn.yelp_db
yelp_db

Database(MongoClient('localhost', 27017), u'yelp_db')

Print out the databases we have so far. `yelp_db` was not added to our list of databases because we haven't added any data to it yet. So let's add some data.

In [3]:
conn.database_names()

[u'local']

We'll create a collection in our `yelp_db` database of all the review data we have.

In [5]:
reviews = yelp_db.reviews
reviews

Collection(Database(MongoClient('localhost', 27017), u'yelp_db'), u'reviews')

In [6]:
yelp_db.collection_names()

[]

Again, the collection will not show up until we actually add some data to it.

Let's add the `mango.json` data to our database. Each line of the JSON file is one JSON Object. We will first parse this data from it's string format to a Python dictionary and then insert this dictionary into the `reviews` collection. MongoDB will handle the insertion process for us.

In [7]:
import json

with open('mango.json', 'r') as reviews_file:
    for line in reviews_file:
        doc = json.loads(line)
        reviews.insert(doc)

Now we will see that the list of databases has our new `yelp_db`.

In [8]:
conn.database_names()

[u'local', u'yelp_db']

And the list of collections in our database has also been updated. Do not worry about the `systems.indexes` collection, it is created and managed by MongoDB.

In [9]:
yelp_db.collection_names()

[u'reviews', u'system.indexes']

## Analyzing the Objects in our Database

Let's see what a sample review object looks like.

In [10]:
reviews.find_one()

{u'_id': ObjectId('56082ad26651fe2b8a7a2597'),
 u'business_id': u'McikHxxEqZ2X0joaRNKlaw',
 u'date': u'2013-06-17',
 u'review_id': u'5XzkZUgPDNZ3WfZss6C-NQ',
 u'stars': 3,
 u'text': u"Nothing special. Just like Applebee's, Chili's or Fridays. Stick to the bar food (burgers, sandwiches, pizza, wings and salads) and you will be okay. \n\nThis was our second attempt at trying Rock Bottom on Fathers Day. The first time almost ten years ago. We went in the early afternoon and they had already ran out of steaks. How do you do that on Fathers Day?! This time we went around 12 noon and beat the rush.\n\nMy wife got and loved the Berry Bash Sangria. She just said it had too much ice in it. Our daughter got the root beer float which she love and I started with a Mango Ice Tea, but later switched for a raspberry ice tea as the mango ice tea wasn't sweet enough.\n\nWe started off the the Kobe beef sliders as our app. It took at least 15 minutes to get them. You get three sliders and they were very

Let's look at the type of this review object so that we can figure out how to interface with it.

In [11]:
type(reviews.find_one())

dict

It is just a regular Python dictionary. MongoDB handled the conversion process for us. Let's look at how many total reviews we have in our database.

In [12]:
reviews.count()

8366

Similarly, we create a collection in our db that will store all the businesses the mango reviews are of.

In [13]:
businesses = yelp_db.businesses

with open('mango_businesses.json', 'r') as biz_file:
    for line in biz_file:
        doc = json.loads(line)
        businesses.insert(doc)

In [14]:
businesses.count()

2984

In [15]:
businesses.find_one()

{u'_id': ObjectId('56082adb6651fe2b8a7a4645'),
 u'attributes': {u'Accepts Credit Cards': True,
  u'Alcohol': u'full_bar',
  u'Ambience': {u'casual': True,
   u'classy': False,
   u'divey': False,
   u'hipster': False,
   u'intimate': False,
   u'romantic': False,
   u'touristy': False,
   u'trendy': False,
   u'upscale': False},
  u'Attire': u'casual',
  u'Caters': False,
  u'Delivery': False,
  u'Good For': {u'breakfast': False,
   u'brunch': False,
   u'dessert': False,
   u'dinner': True,
   u'latenight': False,
   u'lunch': False},
  u'Good For Groups': True,
  u'Good For Kids': False,
  u'Good for Kids': True,
  u'Has TV': True,
  u'Noise Level': u'average',
  u'Outdoor Seating': True,
  u'Parking': {u'garage': False,
   u'lot': True,
   u'street': False,
   u'valet': False,
   u'validated': False},
  u'Price Range': 2,
  u'Take-out': True,
  u'Takes Reservations': True,
  u'Waiter Service': True,
  u'Wheelchair Accessible': True,
  u'Wi-Fi': u'no'},
 u'business_id': u'McikHxxEqZ2

Let's find the number of businesses in Illinois that serve mango. We will query the database, and use a simple query operator `$eq` which matches values that are equal to the specified value.

In [16]:
illinois_businesses = businesses.find( {'state':{'$eq': 'IL'}} )
illinois_businesses.count()

41

In [17]:
type(illinois_businesses)

pymongo.cursor.Cursor

Whenever we run a query, the object that is returned is a `cursor`. Cursors are used to iterate over the data that we queried. We can use it like a list of objects and loop over it.

We can analyze the first object in the cursor by calling `next()` on it.

In [18]:
illinois_businesses.next()

{u'_id': ObjectId('56082adb6651fe2b8a7a466c'),
 u'attributes': {u'Accepts Credit Cards': True,
  u'Alcohol': u'none',
  u'Ambience': {u'casual': True,
   u'classy': False,
   u'divey': False,
   u'hipster': False,
   u'intimate': False,
   u'romantic': False,
   u'touristy': False,
   u'trendy': False,
   u'upscale': False},
  u'Attire': u'casual',
  u'BYOB/Corkage': u'yes_free',
  u'Caters': False,
  u'Corkage': False,
  u'Delivery': False,
  u'Good For': {u'breakfast': False,
   u'brunch': False,
   u'dessert': False,
   u'dinner': True,
   u'latenight': False,
   u'lunch': True},
  u'Good For Groups': True,
  u'Good for Kids': True,
  u'Has TV': False,
  u'Noise Level': u'average',
  u'Outdoor Seating': False,
  u'Parking': {u'garage': False,
   u'lot': False,
   u'street': True,
   u'valet': False,
   u'validated': False},
  u'Price Range': 2,
  u'Take-out': True,
  u'Takes Reservations': False,
  u'Waiter Service': True,
  u'Wheelchair Accessible': False,
  u'Wi-Fi': u'no'},
 u'bu

We can specifically find one document in our collection by specifying it's properties.

In [19]:
bombay_grill = businesses.find_one( {'name':{'$eq': 'Bombay Indian Grill'}, 'state': {'$eq': 'IL'}} )
bombay_grill

{u'_id': ObjectId('56082adb6651fe2b8a7a466c'),
 u'attributes': {u'Accepts Credit Cards': True,
  u'Alcohol': u'none',
  u'Ambience': {u'casual': True,
   u'classy': False,
   u'divey': False,
   u'hipster': False,
   u'intimate': False,
   u'romantic': False,
   u'touristy': False,
   u'trendy': False,
   u'upscale': False},
  u'Attire': u'casual',
  u'BYOB/Corkage': u'yes_free',
  u'Caters': False,
  u'Corkage': False,
  u'Delivery': False,
  u'Good For': {u'breakfast': False,
   u'brunch': False,
   u'dessert': False,
   u'dinner': True,
   u'latenight': False,
   u'lunch': True},
  u'Good For Groups': True,
  u'Good for Kids': True,
  u'Has TV': False,
  u'Noise Level': u'average',
  u'Outdoor Seating': False,
  u'Parking': {u'garage': False,
   u'lot': False,
   u'street': True,
   u'valet': False,
   u'validated': False},
  u'Price Range': 2,
  u'Take-out': True,
  u'Takes Reservations': False,
  u'Waiter Service': True,
  u'Wheelchair Accessible': False,
  u'Wi-Fi': u'no'},
 u'bu

To get the number of mango reviews that people have posted about Bombay Indian Grill, we will query the reviews collection by using the restaurant's `business_id`.

In [20]:
bombay_grill_reviews = reviews.find( {'business_id': {'$eq': bombay_grill['business_id']}} )
bombay_grill_reviews.count()

2

We can print the text of each of it's reviews by iterating through the cursor.

In [21]:
for mango_review in bombay_grill_reviews:
    print mango_review['text']

This is one of my favorite places in CU.  Their Naan is the best Naan of all Naans I've had - and I've had a lot of Naan.  Chicken Tikka and Lamb Curry are my favorites.  With a mango lassi expect to pay around 15 bucks after tax and tip.
Okay Indian food. Bit overpriced. The naan is big and fresh. The service is sub par. I ordered an aloo paratha once and the owner brought me a regular paratha. Did you think I didn't know? The vindaloo tastes okay. The lassi is waaayyy too thick. They do have mango kulfi, which Ambar does not. But 10 out of 10 times I would go to Ambar.


These two reviews seem to have different opinions about the restaurant. We should find the star rating they specified in their review. But before we access our cursor again, we need to rewind it because we have already reached the end of the objects that it iterates over. Rewinding moves the cursor back to the start of the object list.

In [22]:
bombay_grill_reviews.rewind()
for mango_review in bombay_grill_reviews:
    print mango_review['stars']

5
2


## Using Logical Query Operators

We can use logical query operators to apply multiple conditions to our query. For example, let's find out all the mango-serving businesses that have over 50 reviews and also have a delivery service.

In [23]:
awesome_restaurants = businesses.find( {'$and': [ {'review_count': {'$gt': 10}}, {'attributes.Delivery': True} ]} )
awesome_restaurants.count()

266